In [2]:
import os

import kagglehub

# Download latest version
directory = kagglehub.dataset_download("chitwanmanchanda/fraudulent-transactions-data")
path = os.path.join(directory, "Fraud.csv")

/Users/andrewgazelka/Projects/daft-connect-example/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from daft.daft import connect_start
from pyspark.sql import SparkSession

# start Daft Connect (spark-connect compatible server)
handle = connect_start()

port = handle.port()

# create a Spark Connect client connected to the Daft Connect server
spark = SparkSession.builder.remote(f"sc://localhost:{port}").appName("DaftPySparkExample").getOrCreate()

In [7]:
data = [[2021, "test", "Albany", "M", 42]]
columns = ["Year", "First_Name", "County", "Sex", "Count"]

df1 = spark.createDataFrame(data, schema="Year int, First_Name STRING, County STRING, Sex STRING, Count int")
df1.show()

🗡️ 🐟 Limit: 00:00 0 rows received, 0 rows emitted
🗡️ 🐟 InMemory: 00:00 0 rows emitted
╭───────┬──────┬────────┬──────┬───────╮          
│ _1    ┆ _2   ┆ _3     ┆ _4   ┆ _5    │
│ ---   ┆ ---  ┆ ---    ┆ ---  ┆ ---   │
│ Int64 ┆ Utf8 ┆ Utf8   ┆ Utf8 ┆ Int64 │
╞═══════╪══════╪════════╪══════╪═══════╡
│ 2021  ┆ test ┆ Albany ┆ M    ┆ 42    │
╰───────┴──────┴────────┴──────┴───────╯


In [8]:
df = spark.read.csv(path, header=True, inferSchema=True, sep=",")

df.show()

Ignoring is_streaming: false
Ignoring schema: ""; not yet implemented
Ignoring options: {"inferSchema": "True", "header": "True", "sep": ","}; not yet implemented


🗡️ 🐟 Limit: 00:00 0 rows received, 0 rows emitted
🗡️ 🐟 ScanTask: 00:00 0 rows emitted
🗡️ 🐟 ScanTask: 00:00 21 rows emitted
╭───────┬──────────┬───────────┬────────────┬────────────────┬─────────┬────────────────╮
│ step  ┆ type     ┆ amount    ┆      …     ┆ newbalanceDest ┆ isFraud ┆ isFlaggedFraud │
│ ---   ┆ ---      ┆ ---       ┆            ┆ ---            ┆ ---     ┆ ---            │
│ Int64 ┆ Utf8     ┆ Float64   ┆ (5 hidden) ┆ Float64        ┆ Int64   ┆ Int64          │
╞═══════╪══════════╪═══════════╪════════════╪════════════════╪═════════╪════════════════╡
│ 1     ┆ PAYMENT  ┆ 9839.64   ┆ …          ┆ 0              ┆ 0       ┆ 0              │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 1     ┆ PAYMENT  ┆ 1864.28   ┆ …          ┆ 0              ┆ 0       ┆ 0              │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 1     ┆ TRANSFER ┆ 181       ┆ …          ┆ 0              ┆ 1   

In [10]:
# Rename columns
corrected_cols = {
    "oldbalanceOrg": "oldBalanceOrig",
    "newbalanceOrig": "newBalanceOrig",
    "oldbalanceDest": "oldBalanceDest",
    "newbalanceDest": "newBalanceDest",
}
for old_col, new_col in corrected_cols.items():
    df = df.withColumnRenamed(old_col, new_col)

df.show()

SparkConnectGrpcException: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "Error in Daft server: Failed to show string

Caused by:
    Unsupported relation type: WithColumnsRenamed(WithColumnsRenamed { input: Some(Relation { common: Some(RelationCommon { source_info: "", plan_id: Some(14), origin: None }), rel_type: Some(WithColumnsRenamed(WithColumnsRenamed { input: Some(Relation { common: Some(RelationCommon { source_info: "", plan_id: Some(13), origin: None }), rel_type: Some(WithColumnsRenamed(WithColumnsRenamed { input: Some(Relation { common: Some(RelationCommon { source_info: "", plan_id: Some(12), origin: None }), rel_type: Some(WithColumnsRenamed(WithColumnsRenamed { input: Some(Relation { common: Some(RelationCommon { source_info: "", plan_id: Some(11), origin: None }), rel_type: Some(WithColumnsRenamed(WithColumnsRenamed { input: Some(Relation { common: Some(RelationCommon { source_info: "", plan_id: Some(10), origin: None }),

In [10]:
from pyspark.sql import functions as F

# First .where() filter DataFrame and return another DataFrame
# Then .select() select from the returned DataFrame
df.select(df.type, F.col('amount')).show(10)

Ignoring is_streaming: false
Ignoring schema: ""; not yet implemented
Ignoring options: {"sep": ",", "header": "True", "inferSchema": "True"}; not yet implemented
Ignoring is_streaming: false
Ignoring schema: ""; not yet implemented
Ignoring options: {"sep": ",", "header": "True", "inferSchema": "True"}; not yet implemented
Ignoring plan_id 2 for attribute expressions; not yet implemented


🗡️ 🐟 Limit: 00:00 0 rows received, 0 rows emitted
🗡️ 🐟 ScanTask: 00:00 0 rows emitted
╭──────────┬──────────╮             d              d
│ type     ┆ amount   │
│ ---      ┆ ---      │
│ Utf8     ┆ Float64  │
╞══════════╪══════════╡
│ PAYMENT  ┆ 9839.64  │
├╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ PAYMENT  ┆ 1864.28  │
├╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ TRANSFER ┆ 181      │
├╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ CASH_OUT ┆ 181      │
├╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ PAYMENT  ┆ 11668.14 │
├╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ PAYMENT  ┆ 7817.71  │
├╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ PAYMENT  ┆ 7107.77  │
├╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ PAYMENT  ┆ 7861.64  │
├╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ PAYMENT  ┆ 4024.36  │
├╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ DEBIT    ┆ 5337.77  │
╰──────────┴──────────╯
